In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import re

def clear(df):
    
    df = df.dropna()
    df = df.reset_index(drop=True)
    
    # 增加一列 words
    df['words'] = ''   
    

    pattern1='[a-zA-Z0-9]'
    pattern2 = '\[.*?\]'
    pattern3 = re.compile(u'[^\s1234567890:：' + '\u4e00-\u9fa5]+')
    pattern4='[’!"#$%&\'()*+,-./：:;<=>?@[\\]^_`{|}~]+'
    
    for i in range(len(df)):
        
        # 去空格
        df['words'][i] = df['content'][i].replace(' ','')

        line1=re.sub(pattern1,'',df['words'][i])   #去除字母和数字
        line2=re.sub(pattern2,'',line1)            #去除表情
        line3=re.sub(pattern3,'',line2)            #去除其它字符
        line4=re.sub(pattern4, '', line3)          #去掉残留的冒号及其它符号
        new_text=''.join(line4.split())       
        
        df['words'][i] = new_text
    
    return df

In [3]:
from langconv import *

# 将繁体转换成简体
    
def tradition2simple(line):
    
    line = Converter('zh-hans').convert(line)
#     line = line.encode('utf-8')
    
    return line

In [4]:
import jieba
import paddle
paddle.enable_static()

jieba.load_userdict("data/dictionary/user_dict.txt")     # 加入自定义金融词典
jieba.enable_paddle()    

# 去停词
def stopwordslist(filepath):  
    
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
 
def seg_sentence(sentence,use_paddle=False):  
    
    sentence = tradition2simple(sentence)
    
    sentence_seged = jieba.cut(sentence, use_paddle=use_paddle)  
    stopwords = stopwordslist('data/stopwords/user_stopwords.txt')  # 停用词的路径  
    outstr = ''  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += "/"   
    return outstr

def token(df,user_paddle=False):
    
    for i in range(len(df)):
        
        df.words[i] = seg_sentence(df.words[i])                    # 选择精确模式
#         df.words[i] = seg_sentence(df.words[i],use_paddle=True)    # 选择paddle模式
    
    return df

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.624 seconds.
Prefix dict has been built successfully.
Paddle enabled successfully......


In [5]:
def token_file(filename,t1,t2):
    
    df_hs300 = pd.read_csv('data/HS300/' + filename + '.csv')
    stock_code = list(df_hs300['成份代码'])
    foldername = t1+'-'+t2
    
    for i in range(len(stock_code)):
        
        one_stock_code = stock_code[i]
        savefilepath = 'data/HS300/' + foldername + '/'+ one_stock_code+'.csv'
        df = pd.read_csv(savefilepath)
        df = clear(df)
        df = token(df)
        df.to_csv(savefilepath, index=False, encoding='utf_8_sig')

In [6]:
# token_file('2015-12-30','20151230','20160613')

In [7]:
# token_file('2016-06-13','20160613','20161212')
# token_file('2016-12-12','20161212','20170612')

# token_file('2017-06-12','20170612','20171211')
# token_file('2017-12-11','20171211','20180611')

# token_file('2018-06-11','20180611','20181217')
# token_file('2018-12-17' ,'20181217','20190617')

# token_file('2019-06-17','20190617','20191216')
# token_file('2019-12-16','20191216','20200615')

token_file('2020-06-15','20200615','20201214')

In [ ]:
df = pd.read_csv("data/HS300/20151230-20160613/000009.SZ.csv")
df = clear(df)
df = token(df)
df.to_csv(savefilepath, index=False, encoding='utf_8_sig')

In [ ]:
df.words[0]

In [ ]:
df.words[1]

In [ ]:
df.words[2]

In [ ]:
df.words[3]

In [ ]:
df.words[4]

In [ ]:
df.words[5]